# Notebook to load scraped data and clean

In [2]:
import pandas as pd
import numpy as np
import string
import datetime
import re

In [20]:
cols = [
    'product',
    'productCategory',
    'productCategoryDetail',
    'price',
    'averageRating',
    'username',
    'userRating',
    'userLocation',
    'athleteType',
    'ageRange',
    'bodyType',
    'likes',
    'dislikes',
    'fit',
    'reviewTitle',
    'reviewDate',
    'reviewText',
    'responseDate',
    'responseText',
    'Helpful',
    'NotHelpful'
]
df = pd.read_csv('mensrunning_lululemon.csv', names=cols)
pd.options.display.max_colwidth = 100

In [21]:
df.shape

(2418, 21)

In [22]:
df.head()

,product,productCategory,productCategoryDetail,price,averageRating,username,userRating,userLocation,athleteType,ageRange,...,likes,dislikes,fit,reviewTitle,reviewDate,reviewText,responseDate,responseText,Helpful,NotHelpful
0,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,$68.00,2.9 out of 5,NYR26,5 out of 5,"STAMFORD, CT",SWEATY GENERALIST,18-24,...,NaN,NaN,NaN,The best tshirt,2018-10-12,These are the most comfortable T-shirts. Wish you guys would make more colors and bring back the...,NaN,NaN,0,0
1,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,$68.00,2.9 out of 5,MJB23,1 out of 5,"CHICAGO, IL, USA",SWEATY GENERALIST,18-24,...,design,quality,NaN,Very Poor Quality,2018-10-11,I purchased this Short Sleeve expecting it to be just like all the other ones I have in my colle...,"October 11, 2018","Dear mjb23,\n\nThanks for reaching out and providing this feedback for us. We hear where you're ...",0,0
2,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,$68.00,2.9 out of 5,JKD123,1 out of 5,"CHICAGO, IL, USA",RUNNER,25-34,...,NaN,"stretch, comfort",NaN,Stretches When You Sweat,2018-10-08,I bought this shirt for my husband to run the Chicago Marathon. Upon starting the race it fit gr...,"October 9, 2018","Hi Jkd123,\n\nThanks for reaching out and providing this feedback for us. I completely understan...",0,0
3,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,$68.00,2.9 out of 5,SLASH,5 out of 5,"OVERLAND PARK, KS, USA",SWEATY GENERALIST,45-54,...,NaN,NaN,NaN,Love Lulu,2018-10-08,I have several shirts and shorts I have purchased from Lululemon. Hands down the best clothing I...,NaN,NaN,0,0
4,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,$68.00,2.9 out of 5,BRANDONM19,4 out of 5,"HUTCHINSON, KS",SWEATY GENERALIST,18-24,...,sweat wicking material,NaN,NaN,Sweat wicking shirt,2018-10-03,Great for workouts.,NaN,NaN,0,0


In [23]:
## Pre process data ##

# Convert to datetime objects # 
df['reviewDate'] = pd.to_datetime(df['reviewDate'], infer_datetime_format=True)
df['responseDate'] = pd.to_datetime(df['responseDate'], infer_datetime_format=True)

# Strip ratings
df['userRating'] = df['userRating'].apply(lambda x: re.split(' ', x)[0]).map(float)
df['averageRating'] = df['averageRating'].apply(lambda x: re.split(' ', x)[0]).map(float)

# Create columns for length of each review
df['reviewLength'] = df['reviewText'].apply(lambda x: len(x))

# Strip dollar signs
df['price'] = df['price'].str.slice(1).astype(float)

# Create boolean column for response or not
df['response'] = np.where(df['responseText'].notnull(), 1, 0)

In [24]:
df.to_csv('cleaned_mensrunning_lululemon.csv', index=False)

In [12]:
# Create a dataframe of unique products and counts of reviews

userReviewCounts = df.groupby('product').size().sort_values(ascending=False).reset_index(name='counts')
userReviewCounts.head()

,product,counts
0,"Pace Breaker Short\n9""",434
1,"Pace Breaker Short\n9"" Updated",248
2,Metal Vent Tech Hoodie,135
3,Metal Vent Tech Long Sleeve,107
4,Surge Warm 1/2 Zip,102


In [14]:
# Create a dataframe of unique products and counts of responses

luluResponseCounts = df[df['responseText'].notnull()].groupby('product')['response'].size().sort_values(ascending=False).reset_index(name='counts')
luluResponseCounts.head()

,product,counts
0,"Pace Breaker Short\n9""",171
1,"Pace Breaker Short\n9"" Updated",146
2,Metal Vent Tech Hoodie,81
3,"Surge Jogger\n29""",38
4,Purist Cycling Water Bottle\n26 oz,37


In [16]:
grp_prices = df.groupby(['price', 'averageRating'])[['product']].agg('count').sort_values('price', ascending=False).reset_index()

In [17]:
grp_prices = grp_prices.rename(columns = {'product':'count'})

In [18]:
grp_prices['price'] = grp_prices['price'].apply(str)